In [1]:
import binascii, gzip, struct, sys, os, numpy as np
from pyspark import SparkConf, SparkContext

In [2]:
trnim = open('train_images.txt', mode='rb').read()
trnlb = open('train_labels.txt', mode='rb').read()
tstim = open('test_images.txt', mode='rb').read()
tstlb = open('test_labels.txt', mode='rb').read()

In [3]:
print(len(trnim), len(trnlb), len(tstim), len(tstlb))

47040016 60008 7840016 10008


In [4]:
trnx = np.zeros(len(trnim)-16)
trny = np.zeros(len(trnlb)-8)
tstx = np.zeros(len(tstim)-16)
tsty = np.zeros(len(tstlb)-8)

In [5]:
for i in range(16, len(trnim)): trnx[i-16] = str(trnim[i])
for i in range(8, len(trnlb)): trny[i-8] = str(trnlb[i])
for i in range(16, len(tstim)): tstx[i-16] = str(tstim[i])
for i in range(8, len(tstlb)): tsty[i-8] = str(tstlb[i])

In [6]:
trnx = np.reshape(trnx/255, (60000, 784))
trny = np.reshape(trny, (60000, 1))
tstx = np.reshape(tstx/255, (10000, 784))
tsty = np.reshape(tsty, (10000, 1))

In [7]:
trnt = np.append(np.zeros((60000,10)), trnx, axis=1)
trnt[:,0:1] = (trny == 0) + 0
trnt[:,1:2] = (trny == 1) + 0
trnt[:,2:3] = (trny == 2) + 0 
trnt[:,3:4] = (trny == 3) + 0
trnt[:,4:5] = (trny == 4) + 0
trnt[:,5:6] = (trny == 5) + 0
trnt[:,6:7] = (trny == 6) + 0
trnt[:,7:8] = (trny == 7) + 0
trnt[:,8:9] = (trny == 8) + 0
trnt[:,9:10] = (trny == 9) + 0

In [8]:
#SparkContext.setSystemProperty('spark.executor.memory', '4g')
#SparkContext.setSystemProperty('spark.driver.memory', '10g')
#SparkContext.setSystemProperty('spark.driver.maxResultSize', '20g')

memory = '12g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

conf = SparkConf()
sc = SparkContext(conf=conf)
#print(sc._conf.getAll())

points = sc.parallelize(trnt)
iterations = 50; D = 784; C = 10; # C for classes, D for dimensions

# Initialize w to a random value
w = 2 * np.random.ranf(D) - 1 
# print("Initial error: " + str(np.sum(trn0[:,0] == ((trn0[:,1:].dot(w)) >= 0.5))/trn0.shape[0]))

# Compute logistic regression gradient for a matrix of data points
def gradient(X, Y, w):
    #Y = matrix[0]    # point labels (first column of input file)
    X = np.reshape(X, (1, D))   # point coordinates
    return np.ravel((1.0 / (1.0 + np.exp(Y * X.dot(w)))) * Y * X.T)
    
def add(x, y):
    x += y
    return x

for i in range(iterations):
    #exx = np.divide(np.exp(trn0[:,1:].dot(w)),1+np.exp(trn0[:,1:].dot(w)))
    #acc = np.sum(trn0[:,0] == (exx >= 0.5))/trn0.shape[0]
    #err = np.sum(np.abs(trn0[:,0] - exx))
    #print("Iteration %i" % (i + 1), "| Accuracy: %6f" % acc, "| Error: %2f" % err)

    w0 -= points.map(lambda m: gradient(m[10:], m[0], w0)).reduce(add)/60000
    w1 -= points.map(lambda m: gradient(m[10:], m[1], w1)).reduce(add)/60000
    w2 -= points.map(lambda m: gradient(m[10:], m[2], w2)).reduce(add)/60000
    w3 -= points.map(lambda m: gradient(m[10:], m[3], w3)).reduce(add)/60000
    w4 -= points.map(lambda m: gradient(m[10:], m[4], w4)).reduce(add)/60000
    w5 -= points.map(lambda m: gradient(m[10:], m[5], w5)).reduce(add)/60000
    w6 -= points.map(lambda m: gradient(m[10:], m[6], w6)).reduce(add)/60000
    w7 -= points.map(lambda m: gradient(m[10:], m[7], w7)).reduce(add)/60000
    w8 -= points.map(lambda m: gradient(m[10:], m[8], w8)).reduce(add)/60000
    w9 -= points.map(lambda m: gradient(m[10:], m[9], w9)).reduce(add)/60000

exx = np.zeros((60000, 10))
exx[:,0] = np.divide(np.exp(trnx.dot(w0)),1+np.exp(trnx.dot(w0)))
exx[:,1] = np.divide(np.exp(trnx.dot(w1)),1+np.exp(trnx.dot(w1)))
exx[:,2] = np.divide(np.exp(trnx.dot(w2)),1+np.exp(trnx.dot(w2)))
exx[:,3] = np.divide(np.exp(trnx.dot(w3)),1+np.exp(trnx.dot(w3)))
exx[:,4] = np.divide(np.exp(trnx.dot(w4)),1+np.exp(trnx.dot(w4)))
exx[:,5] = np.divide(np.exp(trnx.dot(w5)),1+np.exp(trnx.dot(w5)))
exx[:,6] = np.divide(np.exp(trnx.dot(w6)),1+np.exp(trnx.dot(w6)))
exx[:,7] = np.divide(np.exp(trnx.dot(w7)),1+np.exp(trnx.dot(w7)))
exx[:,8] = np.divide(np.exp(trnx.dot(w8)),1+np.exp(trnx.dot(w8)))
exx[:,9] = np.divide(np.exp(trnx.dot(w9)),1+np.exp(trnx.dot(w9)))

acc = np.sum(trny == np.argmax(exx, axis=1))/trny.shape[0]
print("Training Accuracy: %6f" % acc)
#err = np.sum(np.abs(trn0[:,0] - exx))
#print("Result", "| Accuracy: %6f" % acc, "| Error: %2f" % err)

exx = np.zeros((10000, 10))
exx[:,0] = np.divide(np.exp(tstx.dot(w0)),1+np.exp(tstx.dot(w0)))
exx[:,1] = np.divide(np.exp(tstx.dot(w1)),1+np.exp(tstx.dot(w1)))
exx[:,2] = np.divide(np.exp(tstx.dot(w2)),1+np.exp(tstx.dot(w2)))
exx[:,3] = np.divide(np.exp(tstx.dot(w3)),1+np.exp(tstx.dot(w3)))
exx[:,4] = np.divide(np.exp(tstx.dot(w4)),1+np.exp(tstx.dot(w4)))
exx[:,5] = np.divide(np.exp(tstx.dot(w5)),1+np.exp(tstx.dot(w5)))
exx[:,6] = np.divide(np.exp(tstx.dot(w6)),1+np.exp(tstx.dot(w6)))
exx[:,7] = np.divide(np.exp(tstx.dot(w7)),1+np.exp(tstx.dot(w7)))
exx[:,8] = np.divide(np.exp(tstx.dot(w8)),1+np.exp(tstx.dot(w8)))
exx[:,9] = np.divide(np.exp(tstx.dot(w9)),1+np.exp(tstx.dot(w9)))

acc = np.sum(tsty == np.argmax(exx, axis=1))/tsty.shape[0]
print("Testing Accuracy: %6f" % acc)

sc.stop()


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:67: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Training Accuracy: 0.000000
Testing Accuracy: 1027.595000


In [9]:
exx = np.zeros((60000, 10))
exx[:,0] = np.divide(np.exp(trnx.dot(w0)),1+np.exp(trnx.dot(w0)))
exx[:,1] = np.divide(np.exp(trnx.dot(w1)),1+np.exp(trnx.dot(w1)))
exx[:,2] = np.divide(np.exp(trnx.dot(w2)),1+np.exp(trnx.dot(w2)))
exx[:,3] = np.divide(np.exp(trnx.dot(w3)),1+np.exp(trnx.dot(w3)))
exx[:,4] = np.divide(np.exp(trnx.dot(w4)),1+np.exp(trnx.dot(w4)))
exx[:,5] = np.divide(np.exp(trnx.dot(w5)),1+np.exp(trnx.dot(w5)))
exx[:,6] = np.divide(np.exp(trnx.dot(w6)),1+np.exp(trnx.dot(w6)))
exx[:,7] = np.divide(np.exp(trnx.dot(w7)),1+np.exp(trnx.dot(w7)))
exx[:,8] = np.divide(np.exp(trnx.dot(w8)),1+np.exp(trnx.dot(w8)))
exx[:,9] = np.divide(np.exp(trnx.dot(w9)),1+np.exp(trnx.dot(w9)))

acc = np.sum(trny == np.argmax(exx, axis=1))/trny.shape[0]
print("Training Accuracy: %6f" % acc)


Training Accuracy: 0.000000


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  del sys.path[0]


In [10]:
trny

array([[5.],
       [0.],
       [4.],
       ...,
       [5.],
       [6.],
       [8.]])

In [14]:
np.argmax(exx, axis=1) #exx

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [16]:
exx[0]

array([1.68524859e-096, 1.03264250e-056, 9.57413700e-018, 1.51155052e-100,
       4.97644567e-021, 1.63675720e-081, 7.12526998e-064, 3.35978177e-072,
       4.88470488e-069, 7.66041359e-077])

In [20]:
w1

array([-8.32206848e-01, -8.95130507e-01,  1.05001928e-01, -7.82303404e-01,
        8.67902539e-01, -9.06961999e-01, -1.80719791e-01, -1.09823175e-01,
        4.62781884e-01, -7.15414016e-01, -5.72536368e-01,  5.09465642e-01,
        1.76277284e-01,  3.63134341e-01, -9.80966810e-02, -9.59680721e-01,
        3.20304750e-02, -8.11024519e-01,  8.00086879e-01, -3.72753222e-01,
       -8.38358753e-01,  9.44850940e-01, -6.08057280e-01, -8.59189135e-01,
        6.38451577e-01, -4.20087890e-01, -3.47946847e-01, -4.87051326e-01,
       -8.32235011e-01, -1.39908612e-01,  4.88712326e-01, -3.47684493e-01,
        9.62604830e-01,  5.17769654e-01,  4.96367480e-01,  5.55227091e-01,
       -1.87120759e-01, -7.10601207e-01,  4.50102352e-01,  5.29968642e-01,
       -2.58840780e-01, -9.76396699e-01, -8.92045319e-01, -7.46773093e-02,
       -8.68533849e-01, -8.10014944e-01,  8.82593248e-03,  2.10786031e-01,
       -3.66528828e-01, -6.74849062e-01, -9.33438677e-01, -8.35121844e-01,
       -4.12929880e-01, -

# Miscellaneous Code

In [12]:
#txt = ''
#for i in range(0, 1000):
#    txt += str(testim[i])+', '
#print(txt)

In [13]:
sc.stop()